In [9]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

ja = pd.read_csv("https://raw.githubusercontent.com/Niederb/political_science/master/Daten/Ja.csv", sep=";", na_values=["..."])#, encoding="ascii")
nein = pd.read_csv("https://raw.githubusercontent.com/Niederb/political_science/master/Daten/Nein.csv", sep=";", na_values=["..."])#, encoding="ascii")
stimmberechtigte = pd.read_csv("https://raw.githubusercontent.com/Niederb/political_science/master/Daten/Stimmberechtigte.csv", sep=";")
kantonsnamen = list(ja)[3:]

stimmberechtigte = stimmberechtigte.iloc[1:321, 3:]
#stimmberechtigte = pd.to_numeric(stimmberechtigte)
stimmberechtigte = stimmberechtigte.apply(pd.to_numeric, errors='ignore')
stimmberechtigte_2 = np.mean(stimmberechtigte.as_matrix(), 0)
result = pd.DataFrame({  'Kanton' : kantonsnamen, "Stimmberechtigte" : stimmberechtigte_2 })

ja_schweiz = ja["Schweiz"]
nein_schweiz = nein["Schweiz"]
angenommen = ja_schweiz > nein_schweiz

ja_kantone = ja.iloc[:, 3:]
ja_kantone = np.nan_to_num(ja_kantone.values)
nein_kantone = nein.iloc[:, 3:]
nein_kantone = np.nan_to_num(nein_kantone.values)

kantone = np.sum(ja_kantone, axis=1) > np.sum(nein_kantone, axis=1)

decisions = np.zeros((623, 26))
different = np.zeros((623, 26))
for i in np.arange(0, 25):
    ja_ohne = np.delete(ja_kantone, i, 1)
    nein_ohne = np.delete(nein_kantone, i, 1)
    decision = np.sum(ja_ohne, axis=1) > np.sum(nein_ohne, axis=1)
    decisions[:, i] = decision 
    different[:, i] = (decision != 1.0 * angenommen.values)


influence = np.sum(different, 0)
influence_votes = np.sum(different, 1)

result["Einfluss"] = influence

In [10]:
result = result.sort_values("Einfluss")

bar_layout=dict(
    color='rgb(255, 161, 0)'
)
    
tr = go.Bar(
    x=result["Kanton"],
    y=result["Einfluss"],
    text="",
    name="",
    marker=bar_layout
)

layout = go.Layout(
        title='Anzahl entschiedene Abstimmungen pro Kanton',
        xaxis=dict(title='Kanton'),
        yaxis=dict(title='Anzahl entschiedene Abstimmungen'),
    )
trace = [tr]
fig = go.Figure(data=trace, layout=layout)    
py.iplot(fig, filename='einfluss-kantone-abstimmung') 

In [11]:
trace = go.Scatter(
    x = result["Einfluss"],
    y = result["Stimmberechtigte"],
    mode = 'markers',
    text=result["Kanton"],
    marker = dict(
        size = 20,
        color = 'rgba(255, 161, 0, 1.0)',
        line = dict(
            width = 2,
            )
        ),
)
layout = go.Layout(
    hovermode = 'closest',
    title='Entschiedene Abstimmungen vs Anzahl Stimmberechtigte',
    xaxis=dict(title='Anzahl entschiedene Abstimmungen'),
    yaxis=dict(title='Anzahl Stimmberechtigte (Durchschnitt)')
    )
data = [trace] 
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='influence-size')